In [1]:
# Minimal LSTM 3-class classifier for weekly returns (−1 / 0 / +1)
# ---------------------------------------------------------------
# Requirements:
# pip install numpy pandas scikit-learn tensorflow==2.* (or compatible)

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# -------------------
# 1) Load your data
# -------------------
# Assume you already have a DataFrame `df` indexed by weekly dates like the sample you posted.
# Columns used: 'Last Price','ret_1','p_state0_h1','p_state1_h1','p_state2_h1'
# If your DataFrame variable is named differently, just replace `df` below.

df = pd.read_csv("df.csv")  # <-- your DataFrame here

# Ensure time order
df = df.sort_index()

# -------------------
# 2) Build/verify target
# -------------------
# Target definition:
# next_ret = ret_1 shifted by -1 (next period return)
# if next_ret > +0.5%  ->  +1
# if next_ret < -0.5%  ->  -1
# else                 ->   0

NEUTRAL_BAND = 0.007  # 0.5%
if "target" not in df.columns:
    next_ret = df["ret_1"].shift(-1)
    tgt = np.where(next_ret > NEUTRAL_BAND, 1,
          np.where(next_ret < -NEUTRAL_BAND, -1, 0))
    df["target"] = tgt

# Drop the last row (no label after shift) and any rows with NaNs in features/label
feature_cols = [ "Last Price", "ret_1", "p_state0_h1", "p_state1_h1", "p_state2_h1"]
df = df.dropna(subset=feature_cols + ["target"]).copy()

# Optional: quick class distribution check
print("Class counts (−1, 0, +1):", df["target"].value_counts().sort_index().to_dict())

# -------------------
# 3) Train/Val/Test split (chronological 80/10/10)
# -------------------
n = len(df)
i_train_end = int(0.8 * n)
i_val_end   = int(0.9 * n)

df_train = df.iloc[:i_train_end]
df_val   = df.iloc[i_train_end:i_val_end]
df_test  = df.iloc[i_val_end:]

# -------------------
# 4) Make sequences
# -------------------
def make_sequences(block_df, features, lookback=16):
    """Return X (num_seq, lookback, num_feat), y (num_seq,) aligned for next-step label already in block_df['target']."""
    X_list, y_list = [], []
    X_src = block_df[features].values
    y_src = block_df["target"].values
    for t in range(lookback, len(block_df)):
        X_list.append(X_src[t-lookback:t, :])
        y_list.append(y_src[t])  # label for the last timestep
    X = np.array(X_list, dtype=np.float32)
    y = np.array(y_list, dtype=np.int64)
    return X, y

LOOKBACK = 52  # weeks; change later as you like

# Fit scaler on TRAIN ONLY, then transform splits BEFORE building sequences (no leakage)
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(df_train[feature_cols])

df_train_s = df_train.copy()
df_val_s   = df_val.copy()
df_test_s  = df_test.copy()

df_train_s[feature_cols] = scaler.transform(df_train[feature_cols])
df_val_s[feature_cols]   = scaler.transform(df_val[feature_cols])
df_test_s[feature_cols]  = scaler.transform(df_test[feature_cols])

X_tr, y_tr = make_sequences(df_train_s, feature_cols, LOOKBACK)
X_va, y_va = make_sequences(df_val_s,   feature_cols, LOOKBACK)
X_te, y_te = make_sequences(df_test_s,  feature_cols, LOOKBACK)

print("Shapes:", "X_tr", X_tr.shape, "X_va", X_va.shape, "X_te", X_te.shape)

# -------------------
# 5) Map targets {−1,0,1} -> {0,1,2} for SparseCategoricalCrossentropy
# -------------------
to_idx = {-1: 0, 0: 1, 1: 2}
y_tr_i = np.vectorize(to_idx.get)(y_tr)
y_va_i = np.vectorize(to_idx.get)(y_va)
y_te_i = np.vectorize(to_idx.get)(y_te)

# -------------------
# 6) Simple LSTM model
# -------------------
tf.random.set_seed(42)
np.random.seed(42)

model = models.Sequential([
    layers.Input(shape=(LOOKBACK, len(feature_cols))),
    layers.LSTM(32),
    layers.Dense(16, activation="relu"),
    layers.Dense(3, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

es = callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(
    X_tr, y_tr_i,
    validation_data=(X_va, y_va_i),
    epochs=50,
    batch_size=32,
    callbacks=[es],
    verbose=1
)

# -------------------
# 7) Evaluate
# -------------------
probs = model.predict(X_te)
y_hat_i = probs.argmax(axis=1)

# Map back {0,1,2} -> {−1,0,1}
from_idx = {0: -1, 1: 0, 2: 1}
y_hat = np.vectorize(from_idx.get)(y_hat_i)
y_true = y_te  # already in {−1,0,1}

print("\nTest accuracy:", accuracy_score(y_true, y_hat))
print("\nClassification report:\n", classification_report(y_true, y_hat, digits=3))
print("\nConfusion matrix (rows=true, cols=pred classes in order [-1,0,1]):\n",
      confusion_matrix(y_true, y_hat, labels=[-1,0,1]))


Class counts (−1, 0, +1): {-1: 718, 0: 239, 1: 791}
Shapes: X_tr (1346, 52, 5) X_va (123, 52, 5) X_te (123, 52, 5)
Epoch 1/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.4472 - loss: 1.0564 - val_accuracy: 0.4715 - val_loss: 0.9924
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.4775 - loss: 0.9836 - val_accuracy: 0.5285 - val_loss: 0.9848
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.4698 - loss: 0.9794 - val_accuracy: 0.5285 - val_loss: 0.9799
Epoch 4/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.4744 - loss: 0.9779 - val_accuracy: 0.5285 - val_loss: 0.9756
Epoch 5/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.4833 - loss: 0.9775 - val_accuracy: 0.5447 - val_loss: 0.9727
Epoch 6/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.4792 - loss: 0.9760 - val_accuracy: 0.5366 - val_loss: 0.9690
Epoch 7/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.4882 - loss: 0.9752 - val_accuracy: 0.5285 - val_loss: 0.9

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [2]:
print(df['target'].value_counts())

target
 1    791
-1    718
 0    239
Name: count, dtype: int64


In [20]:
from sklearn.linear_model import LogisticRegression

# Use last row of features only (no sequence)
X_last = df[["p_state0_h1","p_state1_h1","p_state2_h1"]].iloc[:-1].values
y_last = df["target"].iloc[1:].values   # shift to next return

logreg = LogisticRegression(max_iter=1000).fit(X_last, y_last)
print("LogReg synthetic acc:", logreg.score(X_last, y_last))

LogReg synthetic acc: 0.4642243846594161


In [18]:
# LSTM with class weights (and optional oversampling for class 0)
# ---------------------------------------------------------------
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# ====== 0) CONFIG ======
FEATURES     = [ "Last Price", "p_state0_h1"]  #, "p_state0_h1", "p_state1_h1", "p_state2_h1"]
LOOKBACK     = 52
NEUTRAL_BAND = 0.005
EPOCHS       = 60
BATCH_SIZE   = 32
USE_OVERSAMPLING = False   # set True to try oversampling class 0 sequences

# ====== 1) PREP ======
df = df.sort_index().copy()

if "target" not in df.columns:
    next_ret = df["ret_1"].shift(-1)
    df["target"] = np.where(next_ret >  NEUTRAL_BAND,  1,
                    np.where(next_ret < -NEUTRAL_BAND, -1, 0))

df = df.dropna(subset=FEATURES + ["target"]).copy()

print("Class counts in FULL data:", df["target"].value_counts().sort_index().to_dict())

# Chronological 80/10/10 split
n = len(df)
i_train_end = int(0.9 * n)
i_val_end   = int(0.95 * n)
df_train, df_val, df_test = df.iloc[:i_train_end], df.iloc[i_train_end:i_val_end], df.iloc[i_val_end:]

# Scale on TRAIN only
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(df_train[FEATURES])
for d in (df_train, df_val, df_test):
    d[FEATURES] = scaler.transform(d[FEATURES])

# ====== 2) SEQUENCING ======
def make_sequences(block_df, features, lookback=16):
    X_list, y_list = [], []
    X_src = block_df[features].values
    y_src = block_df["target"].values
    for t in range(lookback, len(block_df)):
        X_list.append(X_src[t-lookback:t, :])
        y_list.append(y_src[t])
    return np.array(X_list, dtype=np.float32), np.array(y_list, dtype=np.int64)

X_tr, y_tr = make_sequences(df_train, FEATURES, LOOKBACK)
X_va, y_va = make_sequences(df_val,   FEATURES, LOOKBACK)
X_te, y_te = make_sequences(df_test,  FEATURES, LOOKBACK)

print("Shapes:", "X_tr", X_tr.shape, "X_va", X_va.shape, "X_te", X_te.shape)

# Map {-1,0,1} -> {0,1,2}
to_idx   = {-1:0, 0:1, 1:2}
from_idx = {0:-1, 1:0, 2:1}
y_tr_i = np.vectorize(to_idx.get)(y_tr)
y_va_i = np.vectorize(to_idx.get)(y_va)
y_te_i = np.vectorize(to_idx.get)(y_te)

# ====== 3) (Optional) Oversampling of class 0 sequences in TRAIN ======
if USE_OVERSAMPLING:
    # gather indices by class
    idx_m1 = np.where(y_tr_i == 0)[0]
    idx_0  = np.where(y_tr_i == 1)[0]
    idx_p1 = np.where(y_tr_i == 2)[0]
    # target count ~ min(max of the other classes, 0.8 * median) to avoid huge duplication
    target_0 = max(len(idx_m1), len(idx_p1))
    if len(idx_0) > 0 and target_0 > len(idx_0):
        add = np.random.choice(idx_0, size=target_0 - len(idx_0), replace=True)
        X_tr = np.concatenate([X_tr, X_tr[add]], axis=0)
        y_tr_i = np.concatenate([y_tr_i, y_tr_i[add]], axis=0)
        # shuffle after oversampling
        perm = np.random.permutation(len(y_tr_i))
        X_tr, y_tr_i = X_tr[perm], y_tr_i[perm]
        print(f"Oversampled class 0 sequences from {len(idx_0)} to {target_0}")

# ====== 4) Class weights (computed on TRAIN after optional oversampling) ======
classes = np.array([0,1,2])
class_weights_vec = compute_class_weight(class_weight="balanced", classes=classes, y=y_tr_i)
class_weights = {int(c): float(w) for c, w in zip(classes, class_weights_vec)}
print("Class weights (for indices {0,1,2} ≡ {-1,0,1}):", class_weights)

# ====== 5) Model ======
tf.random.set_seed(42)
np.random.seed(42)

model = models.Sequential([
    layers.Input(shape=(LOOKBACK, len(FEATURES))),
    layers.LSTM(32, dropout=0.1, recurrent_dropout=0.0),
    layers.Dense(16, activation="relu"),
    layers.Dense(3, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

es = callbacks.EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True)

history = model.fit(
    X_tr, y_tr_i,
    validation_data=(X_va, y_va_i),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[es],
    verbose=1,
    class_weight=class_weights
)

# ====== 6) Evaluate ======
probs   = model.predict(X_te)
y_hat_i = probs.argmax(axis=1)
y_hat   = np.vectorize(from_idx.get)(y_hat_i)

print("\nTest accuracy:", accuracy_score(y_te, y_hat))
print("Macro F1:", f1_score(y_te, y_hat, labels=[-1,0,1], average="macro"))
print("\nClassification report:\n", classification_report(y_te, y_hat, digits=3))
print("Confusion matrix (rows=true, cols=pred [-1,0,1]):\n",
      confusion_matrix(y_te, y_hat, labels=[-1,0,1]))


Class counts in FULL data: {-1: 718, 0: 239, 1: 791}
Shapes: X_tr (1521, 52, 2) X_va (35, 52, 2) X_te (36, 52, 2)
Class weights (for indices {0,1,2} ≡ {-1,0,1}): {0: 0.8203883495145631, 1: 2.485294117647059, 2: 0.7253218884120172}
Epoch 1/60


/tmp/ipython-input-3311192286.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[FEATURES] = scaler.transform(d[FEATURES])
/tmp/ipython-input-3311192286.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[FEATURES] = scaler.transform(d[FEATURES])
/tmp/ipython-input-3311192286.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.3425 - loss: 1.1218 - val_accuracy: 0.1143 - val_loss: 1.2054
Epoch 2/60
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3278 - loss: 1.1154 - val_accuracy: 0.1143 - val_loss: 1.1771
Epoch 3/60
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3587 - loss: 1.1136 - val_accuracy: 0.1143 - val_loss: 1.1804
Epoch 4/60
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3430 - loss: 1.1093 - val_accuracy: 0.1143 - val_loss: 1.1395
Epoch 5/60
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3514 - loss: 1.1173 - val_accuracy: 0.1143 - val_loss: 1.1170
Epoch 6/60
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.3719 - loss: 1.1180 - val_accuracy: 0.1143 - val_loss: 1.1253
Epoch 7/60
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.3636 - loss: 1.1122 - val_accuracy: 0.1143 - val_loss: 1.1296
Epoch 8/60
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3743 - loss: 1.1137 - val_accuracy: 0.3143 - val_loss: 1.

In [19]:
# ==============================================================
# Feature-subset search (require predicting ALL 3 classes on TEST)
# ==============================================================

import itertools
from collections import OrderedDict

REQUIRE_ALL_CLASSES = True
ALL_CLASSES_SET = {-1, 0, 1}

CANDIDATE_FEATURES = ["Last Price", "ret_1", "p_state0_h1", "p_state1_h1", "p_state2_h1"]

# Ensure target exists and data sorted
df = df.sort_index().copy()
if "target" not in df.columns:
    next_ret = df["ret_1"].shift(-1)
    df["target"] = np.where(next_ret >  NEUTRAL_BAND,  1,
                    np.where(next_ret < -NEUTRAL_BAND, -1, 0))

# Keep rows usable for all features (safe superset)
df = df.dropna(subset=CANDIDATE_FEATURES + ["target"]).copy()

# Chronological 80/10/10 split (fixed for all subsets)
n = len(df)
i_train_end = int(0.8 * n)
i_val_end   = int(0.9 * n)
df_train, df_val, df_test = df.iloc[:i_train_end].copy(), df.iloc[i_train_end:i_val_end].copy(), df.iloc[i_val_end:].copy()

def make_sequences(block_df, features, lookback=16):
    X_list, y_list = [], []
    X_src = block_df[features].values
    y_src = block_df["target"].values
    for t in range(lookback, len(block_df)):
        X_list.append(X_src[t-lookback:t, :])
        y_list.append(y_src[t])
    return np.array(X_list, dtype=np.float32), np.array(y_list, dtype=np.int64)

to_idx   = {-1:0, 0:1, 1:2}
from_idx = {0:-1, 1:0, 2:1}

def build_model(input_dim):
    tf.random.set_seed(42)
    np.random.seed(42)
    m = models.Sequential([
        layers.Input(shape=(LOOKBACK, input_dim)),
        layers.LSTM(32, dropout=0.1, recurrent_dropout=0.0),
        layers.Dense(16, activation="relu"),
        layers.Dense(3, activation="softmax")
    ])
    m.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"]
    )
    return m

def fit_eval_for_features(feat_subset):
    # 1) Scale per subset (fit on TRAIN only)
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler.fit(df_train[feat_subset])
    dtr = df_train.copy(); dva = df_val.copy(); dte = df_test.copy()
    dtr[feat_subset] = scaler.transform(dtr[feat_subset])
    dva[feat_subset] = scaler.transform(dva[feat_subset])
    dte[feat_subset] = scaler.transform(dte[feat_subset])

    # 2) Make sequences
    X_tr, y_tr = make_sequences(dtr, feat_subset, LOOKBACK)
    X_va, y_va = make_sequences(dva, feat_subset, LOOKBACK)
    X_te, y_te = make_sequences(dte, feat_subset, LOOKBACK)

    # Map labels {-1,0,1} -> {0,1,2}
    y_tr_i = np.vectorize(to_idx.get)(y_tr)
    y_va_i = np.vectorize(to_idx.get)(y_va)
    y_te_i = np.vectorize(to_idx.get)(y_te)

    # 3) Optional oversampling (class 0 index = 1)
    X_train_fin, y_train_fin = X_tr, y_tr_i
    if USE_OVERSAMPLING:
        idx_m1 = np.where(y_tr_i == 0)[0]
        idx_0  = np.where(y_tr_i == 1)[0]
        idx_p1 = np.where(y_tr_i == 2)[0]
        target_0 = max(len(idx_m1), len(idx_p1))
        if len(idx_0) > 0 and target_0 > len(idx_0):
            add = np.random.choice(idx_0, size=target_0 - len(idx_0), replace=True)
            X_train_fin = np.concatenate([X_tr, X_tr[add]], axis=0)
            y_train_fin = np.concatenate([y_tr_i, y_tr_i[add]], axis=0)
            perm = np.random.permutation(len(y_train_fin))
            X_train_fin, y_train_fin = X_train_fin[perm], y_train_fin[perm]

    # 4) Class weights from TRAIN
    classes = np.array([0,1,2])
    cw_vec = compute_class_weight(class_weight="balanced", classes=classes, y=y_train_fin)
    class_weights = {int(c): float(w) for c, w in zip(classes, cw_vec)}

    # 5) Build, fit
    model = build_model(input_dim=len(feat_subset))
    es = callbacks.EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True)
    model.fit(
        X_train_fin, y_train_fin,
        validation_data=(X_va, y_va_i),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[es],
        verbose=0,
        class_weight=class_weights
    )

    # 6) Evaluate on TEST
    probs   = model.predict(X_te, verbose=0)
    y_hat_i = probs.argmax(axis=1)
    y_hat   = np.vectorize(from_idx.get)(y_hat_i)

    acc  = accuracy_score(y_te, y_hat)
    f1m  = f1_score(y_te, y_hat, labels=[-1,0,1], average="macro")
    rep  = classification_report(y_te, y_hat, digits=3)
    cm   = confusion_matrix(y_te, y_hat, labels=[-1,0,1])

    pred_classes = set(np.unique(y_hat))
    passes = (pred_classes == ALL_CLASSES_SET) if REQUIRE_ALL_CLASSES else True

    return {
        "features": tuple(feat_subset),
        "n_features": len(feat_subset),
        "test_acc": acc,
        "macro_f1": f1m,
        "report": rep,
        "confusion_matrix": cm,
        "pred_classes": pred_classes,
        "passes_all_classes": passes
    }

# Iterate all non-empty subsets
all_subsets = []
for r in range(1, len(CANDIDATE_FEATURES)+1):
    all_subsets += list(itertools.combinations(CANDIDATE_FEATURES, r))

print(f"Evaluating {len(all_subsets)} feature combinations...")
results, skipped = [], 0
for subset in all_subsets:
    res = fit_eval_for_features(list(subset))
    if REQUIRE_ALL_CLASSES and not res["passes_all_classes"]:
        skipped += 1
        missing = ALL_CLASSES_SET - res["pred_classes"]
        print(f"{subset} -> SKIPPED: missing classes {sorted(list(missing))}  "
              f"(predicted: {sorted(list(res['pred_classes']))})")
        continue
    results.append(res)
    print(f"{subset} -> acc={res['test_acc']:.3f}, f1={res['macro_f1']:.3f} | "
          f"pred_classes={sorted(list(res['pred_classes']))}")

print(f"\nCombinations passing the 'all classes predicted' constraint: {len(results)} "
      f"(skipped {skipped})")

if len(results) == 0:
    print("\nNo combinations satisfied the constraint. "
          "Consider widening NEUTRAL_BAND, enabling USE_OVERSAMPLING, "
          "or increasing EPOCHS/hidden units.")
else:
    results_sorted = sorted(results, key=lambda d: (d["test_acc"], d["macro_f1"]), reverse=True)
    best = results_sorted[0]
    print("\n================ BEST COMBINATION (with all classes predicted) ================")
    print("Features:", best["features"])
    print(f"Test accuracy: {best['test_acc']:.3f}")
    print(f"Macro F1: {best['macro_f1']:.3f}")
    print("Predicted classes:", sorted(list(best["pred_classes"])))
    print("\nClassification report:\n", best["report"])
    print("Confusion matrix (rows=true, cols=pred [-1,0,1]):\n", best["confusion_matrix"])

    # Optional: show top-10 table
    tbl = pd.DataFrame([{
        "features": ", ".join(r["features"]),
        "n_features": r["n_features"],
        "test_acc": r["test_acc"],
        "macro_f1": r["macro_f1"],
        "predicted_classes": "".join(str(c) for c in sorted(list(r["pred_classes"])))
    } for r in results_sorted])
    print("\nTop 10 combinations (constraint-satisfying):")
    print(tbl.head(10).to_string(index=False))


Evaluating 31 feature combinations...


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


('Last Price',) -> SKIPPED: missing classes [0]  (predicted: [np.int64(-1), np.int64(1)])
('ret_1',) -> acc=0.431, f1=0.357 | pred_classes=[np.int64(-1), np.int64(0), np.int64(1)]
('p_state0_h1',) -> acc=0.268, f1=0.264 | pred_classes=[np.int64(-1), np.int64(0), np.int64(1)]
('p_state1_h1',) -> acc=0.309, f1=0.290 | pred_classes=[np.int64(-1), np.int64(0), np.int64(1)]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


('p_state2_h1',) -> SKIPPED: missing classes [-1, 1]  (predicted: [np.int64(0)])


...

('ret_1', 'p_state1_h1', 'p_state2_h1') -> SKIPPED: missing classes [-1]  (predicted: [np.int64(0), np.int64(1)])


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


('p_state0_h1', 'p_state1_h1', 'p_state2_h1') -> SKIPPED: missing classes [-1]  (predicted: [np.int64(0), np.int64(1)])
('Last Price', 'ret_1', 'p_state0_h1', 'p_state1_h1') -> acc=0.504, f1=0.381 | pred_classes=[np.int64(-1), np.int64(0), np.int64(1)]
('Last Price', 'ret_1', 'p_state0_h1', 'p_state2_h1') -> acc=0.431, f1=0.392 | pred_classes=[np.int64(-1), np.int64(0), np.int64(1)]
('Last Price', 'ret_1', 'p_state1_h1', 'p_state2_h1') -> acc=0.382, f1=0.359 | pred_classes=[np.int64(-1), np.int64(0), np.int64(1)]
('Last Price', 'p_state0_h1', 'p_state1_h1', 'p_state2_h1') -> acc=0.447, f1=0.321 | pred_classes=[np.int64(-1), np.int64(0), np.int64(1)]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


('ret_1', 'p_state0_h1', 'p_state1_h1', 'p_state2_h1') -> SKIPPED: missing classes [-1]  (predicted: [np.int64(0), np.int64(1)])
('Last Price', 'ret_1', 'p_state0_h1', 'p_state1_h1', 'p_state2_h1') -> acc=0.350, f1=0.338 | pred_classes=[np.int64(-1), np.int64(0), np.int64(1)]

Combinations passing the 'all classes predicted' constraint: 22 (skipped 9)

================ BEST COMBINATION (with all classes predicted) ================
Features: ('Last Price', 'ret_1', 'p_state0_h1', 'p_state1_h1')
Test accuracy: 0.504
Macro F1: 0.381
Predicted classes: [np.int64(-1), np.int64(0), np.int64(1)]

Classification report:
               precision    recall  f1-score   support

          -1      0.500     0.750     0.600        56
           0      0.200     0.059     0.091        17
           1      0.559     0.380     0.452        50

    accuracy                          0.504       123
   macro avg      0.420     0.396     0.381       123
weighted avg      0.482     0.504     0.470       123

In [4]:
# ============================
# Synthetic 3-class LSTM test
# ============================
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

np.random.seed(7)
tf.random.set_seed(7)

# -------- Config knobs --------
N              = 2500          # number of weekly points
LOOKBACK       = 16
NEUTRAL_BAND   = 0.005         # ±0.5% neutral
# Regime return params (mean, std) weekly
MU   = np.array([-0.012, 0.0, 0.012])   # down, neutral, up
SIG  = np.array([ 0.02 , 0.005, 0.02 ])
# Sticky transition matrix (rows=sum=1)
Tmat = np.array([
    [0.92, 0.06, 0.02],
    [0.05, 0.90, 0.05],
    [0.02, 0.06, 0.92]
])
# How informative the prob features are:
PROB_NOISE_STD = 0.10  # 0.0 = perfect one-hot; higher = noisier/less reliable
PRICE_START    = 3.00

# -------- 1) Generate hidden states --------
states = np.zeros(N, dtype=int)
states[0] = np.random.choice([0,1,2])  # 0=down,1=neutral,2=up
for t in range(1, N):
    states[t] = np.random.choice([0,1,2], p=Tmat[states[t-1]])

# -------- 2) Generate returns conditional on state --------
# ret_t ~ Normal(mu[state_t], sigma[state_t])
ret = np.random.normal(MU[states], SIG[states])
# Build price (weekly)
price = np.empty(N)
price[0] = PRICE_START
for t in range(1, N):
    price[t] = price[t-1] * (1.0 + ret[t])

# -------- 3) Create noisy "predicted" state probabilities (features) --------
# Start with one-hot for true state and add Gaussian noise, then renormalize
probs = np.zeros((N, 3))
probs[np.arange(N), states] = 1.0
probs = probs + np.random.normal(0, PROB_NOISE_STD, size=probs.shape)
probs = np.clip(probs, 1e-6, None)
probs = probs / probs.sum(axis=1, keepdims=True)

# -------- 4) Assemble DataFrame & labels --------
df_syn = pd.DataFrame({
    "Last Price": price,
    "ret_1": ret,
    "p_state0_h1": probs[:,0],
    "p_state1_h1": probs[:,1],
    "p_state2_h1": probs[:,2],
}, index=pd.date_range("1990-01-07", periods=N, freq="W-SUN"))

# label uses NEXT period return
next_ret = df_syn["ret_1"].shift(-1)
target = np.where(next_ret >  NEUTRAL_BAND,  1,
         np.where(next_ret < -NEUTRAL_BAND, -1, 0))
df_syn["target"] = target
df_syn = df_syn.dropna().copy()  # drop last row with no next_ret

print("Synthetic class counts:", df_syn["target"].value_counts().sort_index().to_dict())

# -------- 5) Chronological 80/10/10 split --------
n = len(df_syn)
i_train_end = int(0.8 * n)
i_val_end   = int(0.9 * n)
df_tr, df_va, df_te = df_syn.iloc[:i_train_end], df_syn.iloc[i_train_end:i_val_end], df_syn.iloc[i_val_end:]

FEATURES = ["Last Price", "ret_1", "p_state0_h1", "p_state1_h1", "p_state2_h1"]

# Scale on TRAIN only
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(df_tr[FEATURES])
for d in (df_tr, df_va, df_te):
    d[FEATURES] = scaler.transform(d[FEATURES])

# -------- 6) Make sequences --------
def make_sequences(block_df, features, lookback=16):
    X_list, y_list = [], []
    X = block_df[features].values
    y = block_df["target"].values
    for t in range(lookback, len(block_df)):
        X_list.append(X[t-lookback:t, :])
        y_list.append(y[t])
    return np.array(X_list, dtype=np.float32), np.array(y_list, dtype=np.int64)

X_tr, y_tr = make_sequences(df_tr, FEATURES, LOOKBACK)
X_va, y_va = make_sequences(df_va, FEATURES, LOOKBACK)
X_te, y_te = make_sequences(df_te, FEATURES, LOOKBACK)

# Map {−1,0,1} ↦ {0,1,2}
to_idx   = {-1:0, 0:1, 1:2}
from_idx = {0:-1, 1:0, 2:1}
y_tr_i = np.vectorize(to_idx.get)(y_tr)
y_va_i = np.vectorize(to_idx.get)(y_va)
y_te_i = np.vectorize(to_idx.get)(y_te)

# -------- 7) Simple LSTM model --------
model = models.Sequential([
    layers.Input(shape=(LOOKBACK, len(FEATURES))),
    layers.LSTM(32),
    layers.Dense(16, activation="relu"),
    layers.Dense(3, activation="softmax")
])
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=["accuracy"])
es = callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)

history = model.fit(
    X_tr, y_tr_i,
    validation_data=(X_va, y_va_i),
    epochs=50,
    batch_size=32,
    callbacks=[es],
    verbose=0
)

# -------- 8) Evaluate --------
probs_hat = model.predict(X_te, verbose=0)
y_hat_i = probs_hat.argmax(axis=1)
y_hat = np.vectorize(from_idx.get)(y_hat_i)

print("\nSynthetic Test accuracy:", accuracy_score(y_te, y_hat))
print("Synthetic Macro F1:", f1_score(y_te, y_hat, labels=[-1,0,1], average="macro"))
print("\nClassification report (synthetic):\n", classification_report(y_te, y_hat, digits=3))
print("Confusion matrix (synthetic, rows=true, cols=pred [-1,0,1]):\n",
      confusion_matrix(y_te, y_hat, labels=[-1,0,1]))


Synthetic class counts: {-1: 775, 0: 972, 1: 753}


/tmp/ipython-input-3675114581.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[FEATURES] = scaler.transform(d[FEATURES])
/tmp/ipython-input-3675114581.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[FEATURES] = scaler.transform(d[FEATURES])
/tmp/ipython-input-3675114581.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d


Synthetic Test accuracy: 0.5170940170940171
Synthetic Macro F1: 0.5184062159895805

Classification report (synthetic):
               precision    recall  f1-score   support

          -1      0.529     0.561     0.544        66
           0      0.529     0.500     0.514        90
           1      0.494     0.500     0.497        78

    accuracy                          0.517       234
   macro avg      0.517     0.520     0.518       234
weighted avg      0.517     0.517     0.517       234

Confusion matrix (synthetic, rows=true, cols=pred [-1,0,1]):
 [[37 18 11]
 [16 45 29]
 [17 22 39]]


In [28]:
# ======================================================
# 0) Imports & helpers
# ======================================================
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

np.random.seed(7)
tf.random.set_seed(7)

def make_sequences(block_df, feature_cols, lookback=16, label_col="target"):
    X_list, y_list = [], []
    X = block_df[feature_cols].values
    y = block_df[label_col].values
    for t in range(lookback, len(block_df)):
        X_list.append(X[t-lookback:t, :])
        y_list.append(y[t])
    return np.array(X_list, dtype=np.float32), np.array(y_list, dtype=np.int64)

def map_labels(y, to_idx={-1:0, 0:1, 1:2}):
    return np.vectorize(to_idx.get)(y)

def inv_map_labels(y_idx, from_idx={0:-1, 1:0, 2:1}):
    return np.vectorize(from_idx.get)(y_idx)

def build_lstm(input_dim, lookback=16, units=32):
    model = models.Sequential([
        layers.Input(shape=(lookback, input_dim)),
        layers.LSTM(units),
        layers.Dense(16, activation="relu"),
        layers.Dense(3, activation="softmax")
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=["accuracy"])
    return model

def fit_eval(X_tr, y_tr_i, X_va, y_va_i, X_te, y_te, use_class_weight=True, epochs=60, batch_size=32):
    # Optional class weights
    cw = None
    if use_class_weight:
        classes = np.array([0,1,2])
        w = compute_class_weight(class_weight="balanced", classes=classes, y=y_tr_i)
        cw = {int(c): float(wi) for c, wi in zip(classes, w)}
        print("Class weights:", cw)

    model = build_lstm(input_dim=X_tr.shape[2], lookback=X_tr.shape[1], units=32)
    es = callbacks.EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True)
    model.fit(X_tr, y_tr_i,
              validation_data=(X_va, y_va_i),
              epochs=epochs,
              batch_size=batch_size,
              callbacks=[es],
              verbose=0,
              class_weight=cw)

    probs = model.predict(X_te, verbose=0)
    y_hat_i = probs.argmax(axis=1)
    y_hat   = inv_map_labels(y_hat_i)

    print("Test accuracy:", accuracy_score(y_te, y_hat))
    print("Macro F1:", f1_score(y_te, y_hat, labels=[-1,0,1], average="macro"))
    print("\nClassification report:\n", classification_report(y_te, y_hat, digits=3))
    print("Confusion matrix (rows=true, cols=pred [-1,0,1]):\n",
          confusion_matrix(y_te, y_hat, labels=[-1,0,1]))

# ======================================================
# 1) SYNTHETIC CHECK — PERFECT SIGNAL (noise = 0.0)
# ======================================================
def run_synthetic_perfect(NEUTRAL_BAND=0.005, LOOKBACK=16, N=2500):
    # Hidden states (sticky)
    Tmat = np.array([[0.92,0.06,0.02],
                     [0.05,0.90,0.05],
                     [0.02,0.06,0.92]])
    MU  = np.array([-0.012, 0.0, 0.012])
    SIG = np.array([ 0.02 , 0.005, 0.02 ])

    states = np.zeros(N, dtype=int)
    states[0] = np.random.choice([0,1,2])
    for t in range(1, N):
        states[t] = np.random.choice([0,1,2], p=Tmat[states[t-1]])

    ret = np.random.normal(MU[states], SIG[states])
    price = np.empty(N); price[0] = 3.00
    for t in range(1, N):
        price[t] = price[t-1] * (1.0 + ret[t])

    # PERFECT probabilities (one-hot), i.e., PROB_NOISE_STD = 0.0
    probs = np.zeros((N,3)); probs[np.arange(N), states] = 1.0

    df_syn = pd.DataFrame({
        "Last Price": price,
        "ret_1": ret,
        "p_state0_h1": probs[:,0],
        "p_state1_h1": probs[:,1],
        "p_state2_h1": probs[:,2],
    }, index=pd.date_range("1990-01-07", periods=N, freq="W-SUN"))

    next_ret = df_syn["ret_1"].shift(-1)
    target = np.where(next_ret >  NEUTRAL_BAND,  1,
             np.where(next_ret < -NEUTRAL_BAND, -1, 0))
    df_syn["target"] = target
    df_syn = df_syn.dropna()

    FEATURES = ["Last Price","ret_1","p_state0_h1","p_state1_h1","p_state2_h1"]
    n = len(df_syn); i_tr, i_va = int(0.8*n), int(0.9*n)
    df_tr, df_va, df_te = df_syn.iloc[:i_tr], df_syn.iloc[i_tr:i_va], df_syn.iloc[i_va:]

    scaler = MinMaxScaler(feature_range=(-1,1))
    scaler.fit(df_tr[FEATURES])
    for d in (df_tr, df_va, df_te):
        d[FEATURES] = scaler.transform(d[FEATURES])

    X_tr, y_tr = make_sequences(df_tr, FEATURES, LOOKBACK)
    X_va, y_va = make_sequences(df_va, FEATURES, LOOKBACK)
    X_te, y_te = make_sequences(df_te, FEATURES, LOOKBACK)

    y_tr_i, y_va_i = map_labels(y_tr), map_labels(y_va)
    print("Synthetic (perfect signal) class counts:", df_syn["target"].value_counts().sort_index().to_dict())
    fit_eval(X_tr, y_tr_i, X_va, y_va_i, X_te, y_te, use_class_weight=False, epochs=40)

print("\n=== Running synthetic perfect-signal check (expect very high accuracy) ===")
run_synthetic_perfect(NEUTRAL_BAND=0.005, LOOKBACK=16, N=2500)

# ======================================================
# 2) REAL DATA — widen neutral band to ±1% and add lags/roll stats
# ======================================================
# NOTE: You must provide your DataFrame `df` with at least:
# ['Last Price','ret_1','p_state0_h1','p_state1_h1','p_state2_h1'], indexed by weekly date.
# Example:
# df = your_dataframe.sort_index()

# ---- UNCOMMENT and point df to your data before running ----
# df = df.sort_index().copy()

def run_real_with_features(df,
                           NEUTRAL_BAND=0.01,   # widened neutral band ±1%
                           LOOKBACK=16,
                           use_class_weight=True):
    df = df.sort_index().copy()

    # Rebuild target from NEXT return with wider neutral band
    next_ret = df["ret_1"].shift(-1)
    df["target"] = np.where(next_ret >  NEUTRAL_BAND,  1,
                    np.where(next_ret < -NEUTRAL_BAND, -1, 0))

    # ========== Feature engineering (no leakage) ==========
    # Simple lags of weekly return
    df["ret_lag1"] = df["ret_1"].shift(1)
    df["ret_lag2"] = df["ret_1"].shift(2)
    df["ret_lag3"] = df["ret_1"].shift(3)
    # Rolling mean/vol over last 4 and 8 weeks (uses only past values)
    df["ret_ma_4"]  = df["ret_1"].rolling(4, min_periods=4).mean()
    df["ret_vol_4"] = df["ret_1"].rolling(4, min_periods=4).std()
    df["ret_ma_8"]  = df["ret_1"].rolling(8, min_periods=8).mean()
    df["ret_vol_8"] = df["ret_1"].rolling(8, min_periods=8).std()

    FEATURES = [
        "Last Price","ret_1", "p_state0_h1","p_state1_h1","p_state2_h1",
        "ret_lag1","ret_lag2","ret_lag3",
        "ret_ma_4","ret_vol_4","ret_ma_8","ret_vol_8"
    ]

    df = df.dropna(subset=FEATURES + ["target"]).copy()

    print("\nReal data class counts with ±1% neutral band:", df["target"].value_counts().sort_index().to_dict())

    # Chronological 80/10/10
    n = len(df); i_tr, i_va = int(0.8*n), int(0.9*n)
    df_tr, df_va, df_te = df.iloc[:i_tr], df.iloc[i_tr:i_va], df.iloc[i_va:]

    # Scale on train only
    scaler = MinMaxScaler(feature_range=(-1,1))
    scaler.fit(df_tr[FEATURES])
    for d in (df_tr, df_va, df_te):
        d[FEATURES] = scaler.transform(d[FEATURES])

    # Sequences
    X_tr, y_tr = make_sequences(df_tr, FEATURES, LOOKBACK)
    X_va, y_va = make_sequences(df_va, FEATURES, LOOKBACK)
    X_te, y_te = make_sequences(df_te, FEATURES, LOOKBACK)

    # Labels to indices
    y_tr_i, y_va_i = map_labels(y_tr), map_labels(y_va)

    # Train & evaluate
    fit_eval(X_tr, y_tr_i, X_va, y_va_i, X_te, y_te,
             use_class_weight=use_class_weight, epochs=60, batch_size=32)

# ---- UNCOMMENT to run on your real DataFrame `df` ----
print("\n=== Running real-data model with wider neutral band + lag/roll features ===")
run_real_with_features(df, NEUTRAL_BAND=0.01, LOOKBACK=52, use_class_weight=True)



=== Running synthetic perfect-signal check (expect very high accuracy) ===
Synthetic (perfect signal) class counts: {-1: 775, 0: 972, 1: 753}


/tmp/ipython-input-2711619866.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[FEATURES] = scaler.transform(d[FEATURES])
/tmp/ipython-input-2711619866.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[FEATURES] = scaler.transform(d[FEATURES])
/tmp/ipython-input-2711619866.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

Test accuracy: 0.5170940170940171
Macro F1: 0.5184062159895805

Classification report:
               precision    recall  f1-score   support

          -1      0.529     0.561     0.544        66
           0      0.529     0.500     0.514        90
           1      0.494     0.500     0.497        78

    accuracy                          0.517       234
   macro avg      0.517     0.520     0.518       234
weighted avg      0.517     0.517     0.517       234

Confusion matrix (rows=true, cols=pred [-1,0,1]):
 [[37 18 11]
 [16 45 29]
 [17 22 39]]

=== Running real-data model with wider neutral band + lag/roll features ===

Real data class counts with ±1% neutral band: {-1: 619, 0: 447, 1: 675}
Class weights: {0: 0.9305555555555556, 1: 1.3576494427558257, 2: 0.8411801632140615}


/tmp/ipython-input-2711619866.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[FEATURES] = scaler.transform(d[FEATURES])
/tmp/ipython-input-2711619866.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[FEATURES] = scaler.transform(d[FEATURES])
/tmp/ipython-input-2711619866.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

Test accuracy: 0.43089430894308944
Macro F1: 0.43011377390462363

Classification report:
               precision    recall  f1-score   support

          -1      0.472     0.378     0.420        45
           0      0.417     0.541     0.471        37
           1      0.410     0.390     0.400        41

    accuracy                          0.431       123
   macro avg      0.433     0.436     0.430       123
weighted avg      0.435     0.431     0.428       123

Confusion matrix (rows=true, cols=pred [-1,0,1]):
 [[17 14 14]
 [ 8 20  9]
 [11 14 16]]


In [25]:
from sklearn.linear_model import LogisticRegression

# Use last row of features only (no sequence)
X_last = df_syn[["p_state0_h1","p_state1_h1","p_state2_h1"]].iloc[:-1].values
y_last = df_syn["target"].iloc[1:].values   # shift to next return

logreg = LogisticRegression(max_iter=1000).fit(X_last, y_last)
print("LogReg synthetic acc:", logreg.score(X_last, y_last))

LogReg synthetic acc: 0.5654261704681873
